## Clone Repository

In [12]:
!git clone https://github.com/piainmaster/ProblematicInternetUse.git

Cloning into 'ProblematicInternetUse'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 24 (delta 5), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (24/24), 15.35 KiB | 7.67 MiB/s, done.
Resolving deltas: 100% (5/5), done.


In [13]:
%cd ProblematicInternetUse

/content/ProblematicInternetUse


## Import Dataset

In [14]:
!pip install -q kaggle # install kaggle api client

In [3]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840625 sha256=7243875fc397ecee7b74f98aae8505060814f1bc020d859342e509d9500bae0b
  Stored in directory: /root/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pyspark


In [15]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [16]:
!chmod 600 ~/.kaggle/kaggle.json

In [17]:
!kaggle competitions download -c child-mind-institute-problematic-internet-use

100% 6.21G/6.21G [00:59<00:00, 129MB/s]
100% 6.21G/6.21G [00:59<00:00, 112MB/s]


In [18]:
!unzip -q child-mind-institute-problematic-internet-use.zip -d /content/dataset

In [30]:
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder.appName('ParquetExample').getOrCreate()

# Read the parquet file
df = spark.read.parquet('/content/dataset/series_train.parquet')

In [31]:
df.show()
df.count()

+----+------------+----------+-----------+------------+---------+-------------+---------+---------------+--------------+-------+-------+-------------------+--------+
|step|           X|         Y|          Z|        enmo|   anglez|non-wear_flag|    light|battery_voltage|   time_of_day|weekday|quarter|relative_date_PCIAT|      id|
+----+------------+----------+-----------+------------+---------+-------------+---------+---------------+--------------+-------+-------+-------------------+--------+
|   0|-0.049986254| 0.9789501|0.062423844| 3.457745E-5|3.6280541|          0.0|      5.0|         4089.0|46320000000000|      4|      3|                3.0|2a88cbe9|
|   1| -0.04910988|0.97909266|0.062864974|4.7858557E-5| 3.681389|          0.0| 9.333333|      4088.1667|46325000000000|      4|      3|                3.0|2a88cbe9|
|   2|-0.049397644| 0.9794555| 0.06318933|         0.0|3.7101982|          0.0|13.666667|      4087.3333|46330000000000|      4|      3|                3.0|2a88cbe9|
|   

314569149

## Push changes to GitHub

In [ ]:
!git config --global user.email "pia.neuwirth@stud.plus.ac.at"
!git config --global user.name "piainmaster"

In [ ]:
!git add Main.ipynb
!git commit -m "Updated the notebook with new changes"

On branch main
Your branch is up to date with 'origin/main'.

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	child-mind-institute-problematic-internet-use.zip

nothing added to commit but untracked files present (use "git add" to track)


In [ ]:
!git push origin main

## Import Data

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import warnings
warnings.filterwarnings('ignore')

CSV DATA

In [ ]:
# Import data sets
train_data = pd.read_csv("/content/dataset/train.csv")
test_data = pd.read_csv("/content/dataset/test.csv")

print("Train data shape:" + str(train_data.shape))
# Check for missing values
print("\nMissing Values in Train Data:")
print(train_data.isnull().sum())

Train data shape:(3960, 82)

Missing Values in Train Data:
id                                           0
Basic_Demos-Enroll_Season                    0
Basic_Demos-Age                              0
Basic_Demos-Sex                              0
CGAS-Season                               1405
                                          ... 
SDS-SDS_Total_Raw                         1351
SDS-SDS_Total_T                           1354
PreInt_EduHx-Season                        420
PreInt_EduHx-computerinternet_hoursday     659
sii                                       1224
Length: 82, dtype: int64


### Parquet Data

Initial unpacking in 10 chunks. Each chunk is > 3GB. The range and filename must be manually changed.

In [ ]:
dataset_folder = '/content/dataset/series_train.parquet'

# Get a list of all subfolders (which contain "id=xxxxx")
subfolders = [f.path for f in os.scandir(dataset_folder) if f.is_dir()]
chunk_size = len(subfolders) // 9
subfolders_parts = [subfolders[i:i + chunk_size] for i in range(0, len(subfolders), chunk_size)]

# If the last chunk is smaller than expected (due to uneven split), it will still work.
print(f"Number of subfolders in each part (except possibly the last): {chunk_size}")
print(f"Number of parts created: {len(subfolders_parts)}")

Number of subfolders in each part (except possibly the last): 110
Number of parts created: 10


In [ ]:
import pandas as pd

# Initialize an empty list to hold DataFrame chunks for each part
full_df_parts = []

# Process the first 5 parts
for idx in range(9, 10):  # to be changed
    subfolders_part = subfolders_parts[idx]
    dataframes_part = []  # Temporary list for the current part's DataFrames

    print(f"Processing part {idx + 1}...")

    # Loop through each subfolder in the current part and read the `.parquet` file
    for subfolder in subfolders_part:
        parquet_file = os.path.join(subfolder, 'part-0.parquet')
        df_chunk = pd.read_parquet(parquet_file)
        dataframes_part.append(df_chunk)

    part_df = pd.concat(dataframes_part, ignore_index=True)
    full_df_parts.append(part_df)
    print(f"Part {idx + 1} loaded and concatenated, shape: {part_df.shape}")

full_df_part1 = pd.concat(full_df_parts, ignore_index=True)
full_df_part1.to_csv('/content/parquet_part-10.csv', index=False) # to be changed
print(f"Saved part {idx + 1} to parquet_part-{idx + 1}.csv.")


Processing part 10...
Part 10 loaded and concatenated, shape: (1912338, 13)
Saved part 10 to parquet_part-10.csv.


Combining of the parquet parts is done locally (once).

In [ ]:
# Import data sets
#train_data = pd.read_parquet("/content/dataset/series_train.parquet")
test_data = pd.read_parquet("/content/dataset/series_test.parquet")

df = test_data
df.to_csv('/content/series_test.csv')

print("Test data shape:" + str(df.shape))
# Check for missing values
print("\nMissing Values in Test Data:")
print(df.isnull().sum())

print(df.head())
print(df.info())

Test data shape:(439726, 14)

Missing Values in Test Data:
step                   0
X                      0
Y                      0
Z                      0
enmo                   0
anglez                 0
non-wear_flag          0
light                  0
battery_voltage        0
time_of_day            0
weekday                0
quarter                0
relative_date_PCIAT    0
id                     0
dtype: int64
   step         X         Y         Z      enmo     anglez  non-wear_flag  \
0     0  0.021536  0.022214 -1.022370  0.022853 -88.280762            0.0   
1     1  0.022005  0.022187 -1.019740  0.020231 -88.241707            0.0   
2     2  0.022240  0.022005 -1.019401  0.019893 -88.170067            0.0   
3     3  0.021589  0.022578 -1.018177  0.018667 -88.250031            0.0   
4     4  0.022005  0.023763 -1.014323  0.016848 -88.130775            0.0   

       light  battery_voltage     time_of_day  weekday  quarter  \
0  53.000000      4188.000000  56940000000000   